# Run a training script as a command job

You can use the Python SDK for Azure Machine Learning to submit scripts as command jobs. By using jobs, you can easily keep track of the input parameters and outputs when training a machine learning model.

## Before you start

You'll need the latest version of the  **azureml-ai-ml** package to run the code in this notebook. Run the cell below to verify that it is installed.

> **Note**:
> If the **azure-ai-ml** package is not installed, run `pip install azure-ai-ml` to install it.

In [1]:
!pip show azure-ai-ml

Name: azure-ai-ml
Version: 1.15.0
Summary: Microsoft Azure Machine Learning Client Library for Python
Home-page: https://github.com/Azure/azure-sdk-for-python
Author: Microsoft Corporation
Author-email: azuresdkengsysadmins@microsoft.com
License: MIT License
Location: /anaconda/envs/azureml_py38/lib/python3.8/site-packages
Requires: opencensus-ext-logging, opencensus-ext-azure, azure-storage-file-datalake, azure-storage-blob, strictyaml, pyjwt, azure-storage-file-share, pydash, jsonschema, marshmallow, azure-common, azure-mgmt-core, colorama, typing-extensions, isodate, pyyaml, azure-core, msrest, tqdm
Required-by: 


In [2]:
#!pip install azure-ai-ml

## Connect to your workspace

With the required SDK packages installed, now you're ready to connect to your workspace.

To connect to a workspace, we need identifier parameters - a subscription ID, resource group name, and workspace name. The resource group name and workspace name are already filled in for you. You only need the subscription ID to complete the command.

To find the necessary parameters, click on the subscription and workspace name at the top right of the Studio. A pane will open on the right.

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> Copy the subscription ID and replace **YOUR-SUBSCRIPTION-ID** with the value you copied. </p>

In [3]:
# enter details of your AML workspace
subscription_id = "{your subscription id}"
resource_group = "{your resource group}"
workspace = "{your aml workspace}"

In [4]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

# get a handle to the workspace
ml_client = MLClient(
    DefaultAzureCredential(), subscription_id, resource_group, workspace
)

## Initiate a command job

Run the cell below to train a classification model to predict diabetes. The model is trained by running the **train-model-parameters.py** script that can be found in the **src** folder. It uses the **diabetes.csv** file as the training data. 

- `code`: specifies the folder that includes the script to run.
- `command`: specifies what to run exactly.
- `environment`: specifies the necessary packages to be installed on the compute before running the command.
- `compute`: specifies the compute to use to run the command.
- `display_name`: the name of the individual job.
- `experiment_name`: the name of the experiment the job belongs to.

Note that the command used to test the script in the terminal is the same as the command in the configuration of the job below. 

In [8]:
import os
current_directory = os.getcwd()
current_directory

'/mnt/batch/tasks/shared/LS_root/mounts/clusters/visualizationdmim-cpu/code/Users/Fuad.Maulana/Volve/training script as command job'

In [ ]:
from azure.ai.ml import command

# configure job

job = command(
    code=".",
    command="python train.py --training_data clean_data.csv --n_estimators 212 --min_samples_split 10 --min_samples_leaf 4",
    environment="AzureML-sklearn-0.24-ubuntu18.04-py37-cpu@latest", # Environment yang ada pada aml
    compute="VisualizationDMIM-cpu",
    display_name="volve-train-script",
    experiment_name="volve-training-simplecase"
    )

# submit job
returned_job = ml_client.create_or_update(job)
aml_url = returned_job.studio_url
print("Monitor your job at", aml_url)